# Recursion

# What is recursion?

Functions can call themselves in their definitions; that's recursion.  It seems like it should be illegal to have a circular definition.  And in fact, it's very easy to create cases where the code will never terminate because of the way it calls itself.

In [ ]:
def bad_recursion():
  print("Bad!")
  bad_recursion()

bad_recursion()

But it's not the case that every self-mention results in infinite recursion.  The recursive call may be dealing with an inherently smaller problem.  If the problem keeps getting smaller until it can be handled with a simple case that isn't recursive, then that's a good recursion.



To use a classic example, we can think about factorial, where 7! = $7 * 6 * 5 * 4 * 3 * 2 * 1$ and similarly for other numbers.  We could define this as $7 * 6!$.  We could define $6!$ as $6 * 5!$.  We could generally define $n!$ as $n * (n-1)!$.  And none of this would be helpful unless we also defined a small base case where $1! = 1$.

In [ ]:
def factorial(n):
  # Omitting checks to make sure we're a natural number, etc
  if n == 1:
    return 1
  return n * factorial(n-1)

print (factorial(4))

Here, the code tries to evaluate 4! by trying to find 3!.  Trying to find 3! causes it to try to find 2!.  Trying to evaluate 2!, it tries to evaluate 1!.  And that is the *base case* and a simple lookup.  It returns, and all the calls return their values on the way back up:  $2*1 = 2, 3*2 = 6, 4*6 = 24$.

In [ ]:
def factorial(n):
  # Omitting checks to make sure we're a natural number, etc
  print(f'Evaluating {n}!')
  if n == 1:
    print('Returning 1')
    return 1
  result = n * factorial(n-1)
  print(f'Returning {result}')
  return result

print (factorial(4))

Here are some similar functions that compute their results recursively.  We can compute a sum of consecutive numbers more or less the same way we compute a factorial, stopping early when we hit the starting number.

In [ ]:
# Sum from m to n
def sum_m_to_n(m, n):
    if n == m:
        return m
    result = n + sum_m_to_n(m, n-1)
    return result

sum_m_to_n(3, 7) # 3 + 4 + 5 + 6 + 7 = 25

In [ ]:
# Sum from m to n
def sum_m_to_n(m, n):
    print(f'Evaluating sum from {m} to {n}')
    if n == m:
        print(f'Returning {m}')
        return m
    result = n + sum_m_to_n(m, n-1)
    print(f'Returning {result}')
    return result

sum_m_to_n(3, 7) # 3 + 4 + 5 + 6 + 7 = 25

A power $a^p$ = $a^{p-1}a$, so it can also be computed recursively.

In [ ]:
def mypow(a, p):
    if p == 0:
        return 1
    result = a * mypow(a, p-1)
    return result

mypow(2,8)

In [ ]:
def mypow(a, p):
    print(f'Evaluating {a}^{p}')
    if p == 0:
        print('Returning 1')
        return 1
    result = a * mypow(a, p-1)
    print(f'Returning {result}')
    return result

mypow(2,8)

In each of these cases, the problem in some way gets "smaller" with each recursive call, and our base case code is waiting to handle the return value when the value gets small enough.  Then each function builds on what it gets back from its recursive call.

Now, let's be honest.  These examples could have been coded iteratively; it's not clear there's a benefit to the recursive approach here.

In [ ]:
def iter_factorial(n):
  running_fact = 1
  for i in range(1,n+1):
    running_fact *= i
  return running_fact
  
print(iter_factorial(4))

But sometimes, recursion maps very well on to problem, such that it's difficult to write the code any other way.  Arguably an example of this is computing the power set, or set of all possible subsets, of a set.  Given an arbitrary "first" element of the set, A, the power set contains all subsets that contain A and all subsets that don't contain A.  For example, the power set containing {'a','b'} is equal to those subsets that contain 'a' ({'a'},{'a','b'}) combined with the subsets that don't contain 'a' ({'b'},empty set).  This leads to the recursive approach of computing the power set without the first element, then computing the overall power set as two copies of that smaller subset, one of which has element A added to everything.

In [ ]:
# Demo with a string representing the set because it's a little cleaner to code
def power_set(setstring):
    if len(setstring) == 0:
        return [""]
    subset_list = []
    smaller_power_set = power_set(setstring[1:])
    # The starting character is either in the subset...
    for substring in smaller_power_set:
        subset_list.append(setstring[0] + substring)
    # ...or not.
    for substring in smaller_power_set:
        subset_list.append(substring)
    return subset_list

power_set("abcd")

# Trees

Often the most concise way of handling a problem involving trees is to use recursion.

Simply counting the nodes in a binary tree is a useful beginning exercise in recursion.  How many nodes are in the tree?  There are the nodes in the left tree, the nodes in the right tree, and the node in the center connecting the two.  Trying to describe how this algorithm works in an iterative way can be done, but the code is much simpler written recursively.

In [ ]:
class BinaryTree:
  def __init__(self, left, right):
    self.leftNode = left # Pass None for no child
    self.rightNode = right # again, None for no child

myTree = BinaryTree(BinaryTree(None,None), BinaryTree(BinaryTree(BinaryTree(None,None), None), None))

In [ ]:
def countNodes(tree):
  if (tree == None):
    return 0
  return 1 + countNodes(tree.leftNode) + countNodes(tree.rightNode)

countNodes(myTree)

The problem gets smaller for each recursive call because the subtree on the left is necessarily smaller than the tree as a whole, and the same goes for the right.  So the algorithm *definitely makes progress* as it reduces the problem to one of smaller size.  That's requirement number one for a successful recursive algorithm; the new problem should be smaller.




And the second requirement is having a base case, without which the algorithm will either crash or go on forever.  Having multiple base cases is possible, too.  That's fine, and it's better to write something that works instead of something slick that may not work all the time.


Another interesting property of recursion is that it requires you really *trust your code*, even if you haven't written it yet.  Believing 100% that the smaller case will work is necessary to moving on to think about how to use that value to compute the larger case.  We have to trust that factorial(n-1) works.  We have to trust that the count of the smaller subtree's nodes will work.


# Induction

That sounds a little like magical thinking, so why *does* a recursive algorithm work?  The answer is that it's a little like mathematical induction.  You may at some point have had to prove something by induction:  you prove the truth of the statement for the base case, then you prove that if it's true for n=i, it's also true for n=i+1.  


So it is with recursion.  The base case needs to make sense, and then, the step that you use to get from the smaller case to the larger case must also make sense.  We have base cases for all the algorithms we've seen so far (i=1 for factorial, empty tree for tree node counting, etc).  And in each case, our logic was rock solid for getting to the next step (multiply by n always makes sense, adding the current node to the two subtrees always makes sense).

When coding or designing an algorithm, it might still feel a little bit magical to *just assume* the algorithm works for smaller inputs.  But as long as the base cases work and the logic is sound for solving bigger problems from smaller ones, the whole thing will work.



# Tree example number two:  traversal to build a set

Here is code that iterates through a tree and puts all the values stored in it into a set.  Sets have a "union" functionality to combine with other sets, so we'll want to union the stuff on the left,
the stuff on the right, and our own data.

In [ ]:
class BinaryTree:
  def __init__(self, left, right, s):
    self.leftNode = left # Pass None for no child
    self.rightNode = right # again, None for no child
    self.s = s # a string

btD = BinaryTree(None,None,"D")
btE = BinaryTree(None,None,"E")
btF = BinaryTree(None,None,"F")
myTree = BinaryTree(BinaryTree(btD,btE, "A"),BinaryTree(btF, None, "C"), "B")

def build_set(node):
    if node == None:
      return set()
    # Recursive idea:  union the stuff from the left
    # with stuff from the right with us
    return set(node.s).union(build_set(node.leftNode)) \
                      .union(build_set(node.rightNode))

print(build_set(myTree))

# Worked exercise:  a palindrome checker



A palindrome is a word that reads the same forwards and backwards.  It stands to reason that a word is a palindrome if its outermost letters match, and the word with those letters removed is also a palindrome.  That's the beginning of a recursive approach.  We then need to think about base cases, what it would look like when the algorithm is reaching the end.  It could end up with an empty string, which we would say is a palindrome, or a single letter, which is also a palindrome.  But we probably only need to return False when we have evidence of a mismatch on the ends.

In [ ]:
def palindrome(s):
  if len(s) == 1 or len(s) == 0:
    return True
  if s[0] != s[-1]:  # [-1] is the last letter
    return False
  return palindrome(s[1:-1])

print(palindrome("ababa"))
print(palindrome("axe"))
print(palindrome("amanaplanacanalpanama"))

# Exercise
Try combining some of the ideas above to write code that takes a BinaryTree (as defined above with a string piece of data) and prints every string in the tree (you pick the order).  How could you change the order?

In [ ]:
def print_tree(my_tree):
  if my_tree is None:
    return
  print(my_tree.s)
  print_tree(my_tree.leftNode)
  print_tree(my_tree.rightNode)
  return

btD = BinaryTree(None,None,"D")
btE = BinaryTree(None,None,"E")
btF = BinaryTree(None,None,"F")
myTree = BinaryTree(BinaryTree(btD,btE, "A"),BinaryTree(btF, None, "C"), "B")
print_tree(myTree)
